In [4]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper
from langchain.document_loaders import WebBaseLoader

llm = ChatOpenAI(temperature=0.1)

class DuckDuckGoSearchToolArgsSchema(BaseModel):
    query: str = Field(description="DuckDuckGo에서 검색할 쿼리")

class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearchTool"
    description = "DuckDuckGo에서 정보를 검색합니다."
    args_schema: Type[DuckDuckGoSearchToolArgsSchema] = DuckDuckGoSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)

class WikipediaSearchToolArgsSchema(BaseModel):
    query: str = Field(description="Wikipedia에서 검색할 쿼리")

class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearchTool"
    description = "Wikipedia에서 정보를 검색합니다."
    args_schema: Type[WikipediaSearchToolArgsSchema] = WikipediaSearchToolArgsSchema

    def _run(self, query):
        wiki = WikipediaAPIWrapper()
        return wiki.run(query)

class WebScrapeToolArgsSchema(BaseModel):
    url: str = Field(description="스크랩할 웹사이트의 URL")

class WebScrapeTool(BaseTool):
    name = "WebScrapeTool"
    description = "웹사이트에서 텍스트를 추출합니다."
    args_schema: Type[WebScrapeToolArgsSchema] = WebScrapeToolArgsSchema

    def _run(self, url):
        try:
            loader = WebBaseLoader(url, headers={"User-Agent": "Mozilla/5.0"})
            docs = loader.load()
            return docs[0].page_content if docs else "No content found."
        except Exception as e:
            return f"웹 스크래핑 중 오류 발생: {e}"

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        DuckDuckGoSearchTool(),
        WikipediaSearchTool(),
        WebScrapeTool(),
    ],
)

prompt = "Find the latest information about OpenAI from Wikipedia and DuckDuckGo, and also scrape the text from https://openai.com. Save all research results to a .txt file."

# 예외처리 추가
try:
    result = agent.invoke(prompt)
    with open("research_result.txt", "w", encoding="utf-8") as f:
        f.write(str(result))
    print("리서치 결과가 research_result.txt 파일에 저장되었습니다.")
except Exception as e:
    print("에이전트 실행 중 오류 발생:", e)



> Entering new AgentExecutor chain...

Invoking: `WikipediaSearchTool` with `{'query': 'OpenAI'}`


Page: OpenAI
Summary: OpenAI, Inc. is an American artificial intelligence (AI) organization founded in December 2015 and headquartered in San Francisco, California. It aims to develop "safe and beneficial" artificial general intelligence (AGI), which it defines as "highly autonomous systems that outperform humans at most economically valuable work". As a leading organization in the ongoing AI boom, OpenAI is known for the GPT family of large language models, the DALL-E series of text-to-image models, and a text-to-video model named Sora. Its release of ChatGPT in November 2022 has been credited with catalyzing widespread interest in generative AI.
The organization has a complex corporate structure. As of April 2025, it is led by the non-profit OpenAI, Inc., registered in Delaware, and has multiple for-profit subsidiaries including OpenAI Holdings, LLC and OpenAI Global, LLC. Microsoft 